In [ ]:
#|default_exp lgb_cv

In [ ]:
#|hide
%load_ext autoreload
%autoreload 2

# LightGBMCV

In [ ]:
#|export
import copy
import os
from concurrent.futures import ThreadPoolExecutor
from functools import partial
from typing import Any, Callable, Dict, List, Optional, Sequence, Tuple, Union

import lightgbm as lgb
import numpy as np
import pandas as pd

from mlforecast import Forecast, TimeSeries
from mlforecast.utils import backtest_splits

In [ ]:
#| exporti
class EarlyStopException(BaseException):
    ...
    
def _mape(y_true, y_pred):
    abs_pct_err = abs(y_true - y_pred) / y_true
    return abs_pct_err.groupby(y_true.index.get_level_values(0), observed=True).mean().mean()

def _rmse(y_true, y_pred):
    sq_err = (y_true - y_pred) ** 2
    return sq_err.groupby(y_true.index.get_level_values(0), observed=True).mean().pow(0.5).mean()

def _update(bst, n):
    for _ in range(n):
        bst.update()

def _predict(ts, bst, valid, h, time_col, dynamic_dfs, predict_fn, **predict_fn_kwargs):
    preds = ts.predict(bst, h, dynamic_dfs, predict_fn, **predict_fn_kwargs).set_index(time_col, append=True)
    preds = preds.join(valid)
    return preds

def _update_and_predict(ts, bst, valid, n, h, time_col, dynamic_dfs, predict_fn, **predict_fn_kwargs):
    _update(bst, n)
    return _predict(ts, bst, valid, h, time_col, dynamic_dfs, predict_fn, **predict_fn_kwargs)

In [ ]:
#| export
class LightGBMCV:
    def __init__(
        self,
        freq: Optional[str] = None,  # pandas offset alias, e.g. D, W, M
        lags: List[int] = [],  # list of lags to use as features
        lag_transforms: Dict[int, List[Tuple]] = {},  # list of transformations to apply to each lag
        date_features: List[str] = [],  # list of names of pandas date attributes to use as features, e.g. dayofweek
        num_threads: int = 1,  # number of threads to use when computing the predictions of each window.
    ):
        self.num_threads = num_threads
        cpu_count = os.cpu_count()
        if cpu_count is None:
            num_cpus = 1
        else:
            num_cpus = cpu_count
        self.bst_threads = num_cpus // num_threads
        self.ts = TimeSeries(freq, lags, lag_transforms, date_features, self.bst_threads)
        
    def _should_stop(self, hist, early_stopping_evals, early_stopping_pct):
        if len(hist) < early_stopping_evals + 1:
            return False
        improvement_pct = 1 - hist[-1][1] / hist[-(early_stopping_evals + 1)][1]
        return improvement_pct < early_stopping_pct

    def fit(
        self,
        data: pd.DataFrame,  # time series
        n_windows: int,  # number of windows to evaluate
        window_size: int,  # test size in each window
        params: Dict[str, Any] = {},  # lightgbm parameters
        id_col: str = 'index',  # column that identifies each serie, can also be the index.
        time_col: str = 'ds',  # column with the timestamps
        target_col: str = 'y',  # column with the series values
        static_features: Optional[List[str]] = None,  # column names of the features that don't change in time
        dropna: bool = True,  # drop rows with missing values created by lags
        keep_last_n: Optional[int] = None,  # keep only this many observations of each serie for computing the updates
        dynamic_dfs: Optional[List[pd.DataFrame]] = None,  # future values for dynamic features
        weights: Sequence[float] = None,  # weight for each window
        eval_every: int = 10,  # number of iterations to train before evaluating the full window
        fit_on_all: bool = False,  # return model fitted on all data
        compute_cv_preds: bool = False,  # compute predictions on all folds using final models
        verbose_eval: bool = True,  # print evaluation metrics
        metric: Union[str, Callable] = 'mape',  # evaluation metric
        early_stopping_evals: int = 2,  # stop if the score doesn't improve in these many evaluations
        early_stopping_pct: float = 0.01,  # score must improve at least in this percentage to keep training
        predict_fn: Optional[Callable] = None,  # custom function to compute predictions
        **predict_fn_kwargs,  # additional arguments passed to predict_fn        
    ):
        if eval_every <= 0:
            raise ValueError(
                "eval_every should be > 0. If you don't want to evaluate the complete horizon use "
                "Forecast.cross_validation instead."
            )
        if weights is None:
            use_weights = np.full(n_windows, 1 / n_windows)        
        elif len(weights) != n_windows:
            raise ValueError('Must specify as many weights as the number of windows')
        else:
            use_weights = np.asarray(weights)
        metric2fn = {'mape': _mape, 'rmse': _rmse}
        if callable(metric):
            metric_fn = metric
            metric_name = 'metric'
        else:
            if metric not in metric2fn:
                raise ValueError(f'{metric} is not one of the implemented metrics: ({", ".join(metric2fn.keys())})')
            metric_fn = metric2fn[metric]
            metric_name = metric            
            
        if id_col != 'index':
            data = data.set_index(id_col)
        
        if np.issubdtype(data['ds'].dtype.type, np.integer):
            freq = 1
        else:
            freq = self.ts.freq
        items = []
        for _, train, valid in backtest_splits(data, n_windows, window_size, freq):
            ts = copy.deepcopy(self.ts)
            prep = ts.fit_transform(train, id_col, time_col, target_col, static_features, dropna, keep_last_n)
            ds = lgb.Dataset(prep.drop(columns=[time_col, target_col]), prep[target_col]).construct()
            bst = lgb.Booster({**params, 'num_threads': self.bst_threads}, ds)
            bst.predict = partial(bst.predict, num_threads=self.bst_threads)
            valid = valid.set_index(time_col, append=True)
            items.append((ts, bst, valid))

        hist = []
        n_iter = lgb.basic._choose_param_value('num_iterations', params, 100)['num_iterations']
        metric_values = np.empty(n_windows)

        if self.num_threads == 1:
            try:
                for i in range(0, n_iter, eval_every):
                    for j, (ts, bst, valid) in enumerate(items):                        
                        preds = _update_and_predict(
                            ts,
                            bst,
                            valid,
                            eval_every,
                            window_size,
                            time_col,
                            dynamic_dfs,
                            predict_fn,
                            **predict_fn_kwargs
                        )
                        metric_values[j] = metric_fn(preds[target_col], preds['Booster'])
                    metric_value = metric_values @ use_weights
                    rounds = eval_every + i
                    hist.append((rounds, metric_value))
                    if verbose_eval:
                        print(f'[{rounds:,d}] {metric_name}: {metric_value:,f}')                
                    if self._should_stop(hist, early_stopping_evals, early_stopping_pct):
                        raise EarlyStopException
            except EarlyStopException:
                print(f'Early stopping at round {rounds:,}')
        else:
            try:
                with ThreadPoolExecutor(self.num_threads) as executor:
                    for i in range(0, n_iter, eval_every):
                        futures = []
                        for ts, bst, valid in items:
                            _update(bst, eval_every)
                            future = executor.submit(
                                _predict,
                                ts,
                                bst,
                                valid,
                                window_size,
                                time_col,
                                dynamic_dfs,
                                predict_fn,
                                **predict_fn_kwargs
                            )
                            futures.append(future)
                        cv_preds = [f.result() for f in futures]
                        metric_values[:] = [metric_fn(preds[target_col], preds['Booster']) for preds in cv_preds]
                        metric_value = metric_values @ use_weights
                        rounds = eval_every + i
                        hist.append((rounds, metric_value))
                        if verbose_eval:
                            print(f'[{rounds:,d}] {metric_name}: {metric_value:,f}')
                        if self._should_stop(hist, early_stopping_evals, early_stopping_pct):
                            raise EarlyStopException
            except EarlyStopException:
                print(f'Early stopping at round {rounds:,}.')
        
        self.cv_models_ = [item[1] for item in items]
        if compute_cv_preds:
            with ThreadPoolExecutor(self.num_threads) as executor:
                futures = []            
                for ts, bst, valid in items:
                    future = executor.submit(
                        _predict,
                        ts,
                        bst,
                        valid,
                        window_size,
                        time_col,
                        dynamic_dfs,
                        predict_fn,
                        **predict_fn_kwargs
                    )
                    futures.append(future)            
                self.cv_preds_ = [f.result() for f in futures]

        if fit_on_all:
            params['num_iterations'] = rounds
            self.fcst = Forecast([])
            self.fcst.ts = self.ts
            self.fcst.models = [lgb.LGBMRegressor(**params)]
            self.fcst.fit(
                data,
                id_col,
                time_col,
                target_col,
                static_features,
                dropna,
                keep_last_n,
            )
        else:
            self.ts._fit(data, id_col, time_col, target_col, static_features, keep_last_n)
        return hist

    def predict(
        self,
        horizon: int,  # number of periods to predict in the future
        dynamic_dfs: Optional[List[pd.DataFrame]] = None,  # future values for dynamic features
        predict_fn: Optional[Callable] = None,  # custom function to compute predictions
        **predict_fn_kwargs,  # additional arguments passed to predict_fn
    ) -> pd.DataFrame:
        """Computes the predictions of the final model trained using all of the data."""        
        if not hasattr(self, 'fcst'):
            raise ValueError('Must call fit with fit_on_all=True before. Did you mean cv_predict?')
        return self.fcst.predict(horizon, dynamic_dfs, predict_fn, **predict_fn_kwargs)
    
    def cv_predict(
        self,
        horizon: int,  # number of periods to predict in the future
        dynamic_dfs: Optional[List[pd.DataFrame]] = None,  # future values for dynamic features
        predict_fn: Optional[Callable] = None,  # custom function to compute predictions
        **predict_fn_kwargs,  # additional arguments passed to predict_fn        
    ) -> pd.DataFrame:
        """Computes the predictions of the models fitted during the CV step."""
        return self.ts.predict(self.cv_models_, horizon)

In [ ]:
from fastcore.test import test_fail
from window_ops.expanding import expanding_mean
from window_ops.rolling import rolling_mean, seasonal_rolling_mean

from mlforecast.utils import generate_daily_series

In [ ]:
data = generate_daily_series(1_000, min_length=500, max_length=1_000)

In [ ]:
config = dict(
    freq='D',
    lags=[7],
    lag_transforms={
        7 : [(rolling_mean, 7)],
        14: [(rolling_mean, 7)],
    },
    num_threads=4,
)
cv = LightGBMCV(**config)
cv.fit(data, n_windows=4, window_size=14, params={'verbosity': -1})

[10] mape: 4.802779
[20] mape: 2.769556
[30] mape: 1.014805
[40] mape: 0.789982
[50] mape: 0.715410
[60] mape: 0.689871
[70] mape: 0.681331
[80] mape: 0.678559
[90] mape: 0.677732
Early stopping at round 90.


[(10, 4.802779209175606),
 (20, 2.769555796385574),
 (30, 1.0148054032228888),
 (40, 0.7899819488940557),
 (50, 0.7154097791759417),
 (60, 0.6898714371663679),
 (70, 0.681331404792988),
 (80, 0.678559226720473),
 (90, 0.677732335081906)]

In [ ]:
cv.cv_predict(14)

,ds,Booster,Booster2,Booster3,Booster4
unique_id,,,,,
id_000,2001-11-03,0.253586,0.248238,0.251411,0.252886
id_000,2001-11-04,1.249829,1.250071,1.249231,1.249613
id_000,2001-11-05,2.250443,2.250281,2.250441,2.249819
id_000,2001-11-06,3.249589,3.249735,3.249947,3.250130
id_000,2001-11-07,4.252215,4.251210,4.252896,4.252406
...,...,...,...,...,...
id_999,2002-08-08,5.249569,5.250838,5.249912,5.249056
id_999,2002-08-09,6.249209,6.249434,6.249346,6.249097
id_999,2002-08-10,0.250684,0.251182,0.252552,0.249360


In [ ]:
test_fail(lambda: cv.predict(1), contains='Must call fit with fit_on_all=True')

In [ ]:
cv2 = LightGBMCV(**config)
cv2.fit(data, n_windows=4, window_size=14, metric='rmse', fit_on_all=True, params={'verbosity': -1})

[10] rmse: 0.927015
[20] rmse: 0.449317
[30] rmse: 0.171730
[40] rmse: 0.152624
[50] rmse: 0.150212
[60] rmse: 0.149881
[70] rmse: 0.149869
Early stopping at round 70.


[(10, 0.9270148870967521),
 (20, 0.44931746152420554),
 (30, 0.17173020966422342),
 (40, 0.15262354479234572),
 (50, 0.15021186993934657),
 (60, 0.14988069283784167),
 (70, 0.149868546603391)]

In [ ]:
cv2.predict(14)

,ds,LGBMRegressor
unique_id,,
id_000,2001-11-03,0.252406
id_000,2001-11-04,1.249564
id_000,2001-11-05,2.249034
id_000,2001-11-06,3.250075
id_000,2001-11-07,4.254464
...,...,...
id_999,2002-08-08,5.249972
id_999,2002-08-09,6.249404
id_999,2002-08-10,0.250580
